In [1]:
import numpy as np
from PIL import Image
from skimage.transform import rescale
from skimage import measure as sk_measure
import itertools
import math
import time
from datetime import datetime as dt
import collections

def get_input(day, split=None, f=str.strip):
    fin = f'dat/2021-{day}.txt'
    if split is None:
        input = open(fin, 'r').readlines()
    else:
        input = open(fin, 'r').read().split(split)
    
    if f is not None:
        input = [f(x) for x in input]
    
    return input

def timestamp(ts):
    date = dt.fromtimestamp(ts)
    return f'[{date.hour:02}:{date.minute:02}:{date.second:02}]'

def output(*args, ts=0):
    out = ' '.join([str(x) for x in args])
    if ts:
        ts = timestamp(ts)
        sz = 76 - len(out) - len(ts)
        if sz > 0:
            out += ' ' * sz
        out += ts
    print(out)

def as_ints(input):    
    nums = []
    for x in input:
        x = [int(c) for c in x]
        nums.append(x)
    return np.asarray(nums)

# Day 1

In [2]:
input = get_input(1, f=int)

def depth_inc(lst, win=1):
    cnt = 0
    for i in range(len(lst) - win):
        if lst[i+win] > lst[i]:
            cnt += 1
    return cnt

output(depth_inc(input), ts=1638334897)
output(depth_inc(input, 3), ts=1638335082)

1688                                                              [05:01:37]
1728                                                              [05:04:42]


# Day 2

In [3]:
input = get_input(2)

def find_pos(lst, use_aim=True):
    depth = pos = aim = 0
    for item in lst:
        cmd, X = item.split()
        X = int(X)
        if cmd == 'forward':
            pos += X
            if use_aim:
                depth += X * aim
        elif cmd == 'down':
            if use_aim:
                aim += X
            else:
                depth += X
        elif cmd == 'up':
            if use_aim:
                aim -= X
            else:
                depth -= X
    return depth * pos

output(find_pos(input, False), ts=1638421338)
output(find_pos(input, True), ts=1638421474)

1670340                                                           [05:02:18]
1954293920                                                        [05:04:34]


# Day 3

In [4]:
input = get_input(3)

cnt = [0] * (len(input[0]))
for x in input:
    for i,v in enumerate(x):
        cnt[i] += int(v)

sz = len(input) / 2
gamma = epsilon = ''
for x in cnt:
    if x > sz:
        gamma += '1'
        epsilon += '0'
    else:
        gamma += '0'
        epsilon += '1'
gamma = int(gamma, 2)
epsilon = int(epsilon, 2)

output(gamma*epsilon, ts=1638508079) #1 3969000


def bin_search(lst, pos, most=True):
    cnt = 0
    ones = []
    zeros = []
    for x in lst:
        if int(x[pos]):
            cnt += 1
            ones.append(x)
        else:
            zeros.append(x)
    
    if cnt >= len(lst) / 2:
        if most:
            out = ones
        else:
            out = zeros
    elif most:
        out = zeros
    else:
        out = ones
    
    if len(out) == 1:
        return int(out[0], 2)
    
    return bin_search(out, pos+1, most)


o2  = bin_search(input, 0, True)
co2 = bin_search(input, 0, False)
output(o2 * co2, ts=1638508559) #2 4267809

3969000                                                           [05:07:59]
4267809                                                           [05:15:59]


# Day 4

In [5]:
input = get_input(4)

calls = [int(x) for x in input[0].split(',')]

boards = []
for i in range(2, len(input)-4, 6):
    b = []
    for j in range(5):
        b.append([int(x) for x in input[i+j].split()])
    boards.append(np.asarray(b))

def bingo(board):
    for row in board:
        if np.sum(row) == -5:
            return True
    for col in board.T:
        if np.sum(col) == -5:
            return True
    return False

def score(board, call):
    board[np.where(board==-1)] = 0
    return np.sum(board) * call

cnt = 0
last = len(boards)
for x in calls:
    hit = False
    for b in boards:
        if x in b:
            b[np.where(b==x)] = -1
            if bingo(b):
                cnt += 1
                if cnt == 1:
                    output(score(b,x), ts=1638594977) #1
                elif cnt == last:
                    output(score(b,x), ts=1638595294) #2
                b[:][:] = -99

8580                                                              [05:16:17]
9576                                                              [05:21:34]


# Day 5

In [6]:
def f(s):
    a, _, b = s.split()
    x1, y1 = [int(i) for i in a.split(',')]
    x2, y2 = [int(i) for i in b.split(',')]  
    return x1, y1, x2, y2

input = get_input(5, f=f)

vents = np.zeros((1000,1000), np.int32)
cnt = np.ones((1000,1000), np.int32)

for x1, y1, x2, y2 in input:
    if x1 == x2:
        if y1 > y2:
            r = range(y2, y1+1)
        else:
            r = range(y1, y2+1)
        for y in r:
            vents[x1][y] += 1
    elif y1 == y2:
        if x1 > x2:
            r = range(x2, x1+1)
        else:
            r = range(x1, x2+1)
        for x in r:
            vents[x][y1] += 1

output(np.sum(cnt[np.where(vents > 1)]), ts=1638681085) #1

for x1, y1, x2, y2 in input:
    if x1 != x2 and y1 != y2:
        if x1 < x2:
            r = range(x1, x2+1)
        else:
            r = range(x1, x2-1, -1)
        
        if y1 < y2:
            f = lambda a,b: a+b
        else:
            f = lambda a,b: a-b
        
        for y,x in enumerate(r):
            vents[x][f(y1,y)] += 1

output(np.sum(cnt[np.where(vents > 1)]), ts=1638682511) #2

7436                                                              [05:11:25]
21104                                                             [05:35:11]


# Day 6

In [7]:
input = get_input(6, split=',', f=int)

fish = collections.Counter(input) # count fish at each time

def epoch(a):
    b = {}              # next epoch
    a[7] = a[7] + a[0]  # reset 0 -> 6 (subtract below)
    b[8] = a[0]         # add new fish (not subtracted)
    for i in range(8):  # subtract 1 from each
        b[i] = a[i+1]
    return b

for day in range(80):
    fish = epoch(fish)

assert sum(fish.values()) == 350149
output(sum(fish.values()), ts=1638767460) #1

for day in range(256-80):
    fish = epoch(fish)
    
assert sum(fish.values()) == 1590327954513
output(sum(fish.values()), ts=1638768282) #2

350149                                                            [05:11:00]
1590327954513                                                     [05:24:42]


# Day 7

In [8]:
input = get_input(7, split=',', f=int)

low_lin = low_geom = 10**20
for i in range(max(input)):
    cost_lin = cost_geom = 0
    for x in input:
        cost_lin += abs(x-i)
        d = abs(x-i)
        cost_geom += (d*d+d)//2
    if cost_lin < low_lin:
        low_lin = cost_lin
    if cost_geom < low_geom:
        low_geom = cost_geom
        
output(low_lin, ts=1638853498)
output(low_geom, ts=1638853947)

356992                                                            [05:04:58]
101268110                                                         [05:12:27]


# Day 8

In [9]:
input = get_input(8)

outs = [x.split('|')[1] for x in input]

cnt = 0
for x in outs:
    for y in x.split():
        if len(y) in [2,3,4,7]:
            cnt += 1

assert cnt == 365
output(cnt, ts=1638940069) #1

cnt = 0
for x in input:
    ins, outs = x.split('|')
    ins = ins.split()
    a = b = c = d = e = f = g = ''
    
    # find easy nums by length
    for y in ins:
        if len(y) == 2:
            one = y
        elif len(y) == 3:
            seven = y
        elif len(y) == 4:
            four = y
        elif len(y) == 7:
            eight = y
    # find three and six using one
    for y in ins:
        if len(y) == 5 and one[0] in y and one[1] in y:
            three = y
        elif len(y) == 6 and not (one[0] in y and one[1] in y):
            six = y
    # a - in seven but not one
    for i in seven:
        if i not in one:
            a = i
    # b - in four but not three
    for i in four:
        if i not in three:
            b = i
    # c - in eight but not six
    for i in eight:
        if i not in six:
            c = i
    # d - in three and four but not one
    # g - in three but not four (and not a)
    for i in three:
        if i in four and i not in one:
            d = i
        elif i not in four and i != a:
            g = i
    # e - in six but not three (and not b)
    # f - in six and one
    for i in six:
        if i not in three and i != b:
            e = i
        elif i in one:
            f = i
    
    if '' in [a,b,c,d,e,f,g]:
        print('shiza', ins)
        continue
    
    out = ''
    for y in outs.split():
        if len(y) == 6 and d not in y:
            out += '0'
        elif len(y) == 2:
            out += '1'
        elif len(y) == 5 and e in y:
            out += '2'
        elif len(y) == 5 and c in y and f in y:
            out += '3'
        elif len(y) == 4:
            out += '4'
        elif len(y) == 5 and b in y:
            out += '5'
        elif len(y) == 6 and c not in y:
            out += '6'
        elif len(y) == 3:
            out += '7'
        elif len(y) == 7:
            out += '8'
        elif len(y) == 6 and e not in y:
            out += '9'
        else:
            print('shiza', y)
    #print(out)
    
    cnt += int(out)

assert cnt == 975706
output(cnt, ts=1638942090) #2

365                                                               [05:07:49]
975706                                                            [05:41:30]


# Day 9

In [10]:
input = get_input(9)

nums = []
for x in input:
    x = [int(c) for c in x]
    nums.append(x)
nums = np.asarray(nums)

w = len(nums[0])
h = len(nums)
cnt = 0
for y,row in enumerate(nums):
    for x,val in enumerate(row):
        if x < w-1 and val >= nums[y][x+1]: continue
        if x > 0 and val >= nums[y][x-1]: continue
        if y < h-1 and val >= nums[y+1][x]: continue
        if y > 0 and val >= nums[y-1][x]: continue
        cnt += val + 1

output(cnt, ts=1639026784) #1

# or...
#from skimage import morphology
#minima = nums[np.where(morphology.local_minima(nums))]
#cnt = sum(minima) + len(minima)

# basins are bordered by 9s or edges, everything else is equiv
nums[np.where(nums < 9)] = 1
# label each basin as a unique group
img = sk_measure.label(nums, connectivity=1)
# mask out the 9s borders to leave just basins
img[np.where(nums == 9)] = 0
# let skimage do the heavy lifting
props = sk_measure.regionprops(img)
# get the largest basins
basins = sorted([p.area for p in props], reverse=True)
out = 1
for b in sorted(basins, reverse=True)[:3]:
    out *= b

assert out == 1317792
output(out, ts=1639028100)

436                                                               [05:13:04]
1317792                                                           [05:35:00]


# Day 10

In [11]:
input = get_input(10)

def points(x):
    if x == ')':return 3
    if x == ']':return 57
    if x == '}':return 1197
    if x == '>':return 25137

def points2(x):
    if x == ')':return 1
    if x == ']':return 2
    if x == '}':return 3
    if x == '>':return 4

closure = {
    '<': '>',
    '[': ']',
    '{': '}',
    '(': ')'
}

errors = 0
total = []
for line in input:
    expected = ''
    score = 0
    for x in line:
        if x in closure:
            expected += closure[x]
        elif x != expected[-1]:
            score += points(x)
            break
        else:
            expected = expected[:-1]
    
    if score > 0:
        errors += score
    else:
        for x in reversed(expected):
            score *= 5
            score += points2(x)
        total.append(score)    

assert errors == 216297
output(errors, ts=1639113303)

assert np.median(total) == 2165057169
output(np.median(total), ts=1639113738)

216297                                                            [05:15:03]
2165057169.0                                                      [05:22:18]


# Day 11

In [12]:
input = as_ints(get_input(11))

def flash(x, y, arr):
    w, h = arr.shape
    if x > 0: arr[y][x-1] += 1
    if x > 0 and y > 0: arr[y-1][x-1] += 1
    if y > 0: arr[y-1][x] += 1
    if y > 0 and x < w-1: arr[y-1][x+1] += 1
    if x < w-1: arr[y][x+1] += 1
    if x < w-1 and y < h-1: arr[y+1][x+1] += 1
    if y < h-1: arr[y+1][x] += 1
    if y < h-1 and x > 0: arr[y+1][x-1] += 1

def octo_step(arr):
    mask = np.zeros(arr.shape, np.int32)
    arr += 1
    hit = True
    while hit:
        hit = False
        flashes = np.where(arr > 9)
        for y,x in zip(flashes[0], flashes[1]):
            if mask[y][x] == 0:
                hit = True
                flash(x, y, arr)
        mask[flashes] = 1
    arr[np.where(mask)] = 0
    #print(arr)
    return np.sum(mask)

cnt = 0
for i in range(100):
    cnt += octo_step(input)
output(cnt) #1

cnt = 100
for i in range(10000):
    cnt += 1
    if octo_step(input) == input.size:
        break
output(cnt) #2

1691
216


# Day 12

In [13]:
input = get_input(12)

input = [x.split('-') for x in input]

def next_cave(cave, path, allow_two):
    p = None
    can_go = False
    can_go = cave.isupper() or cave not in path
    if cave.isupper():
        can_go = True
    elif cave == 'start':
        can_go = False
    elif cave not in path:
        can_go == True
    elif allow_two:
        caves = [x for x in path if x.islower()]
        if max(collections.Counter(caves).values()) == 1:
            can_go = True
    
    if can_go:
        p = path.copy()
        p.append(cave)
    return p

def extend_path(cave, path, allow_two):
    paths = []
    for a,b in input:
        if a == cave:
            p = next_cave(b, path, allow_two)
            if p:
                paths.append(p)
        elif b == cave:
            p = next_cave(a, path, allow_two)
            if p:
                paths.append(p)
    return paths
    
def search(paths, allow_two=False):
    if len(paths) == 0:
        cave = 'start'
        path = [cave]
        return extend_path(cave, path, allow_two)
    
    new_paths = []
    for p in paths:
        cave = p[-1]
        if cave == 'end':
            new_paths.append(p)
        else:
            new_paths.extend(extend_path(cave, p, allow_two))
    return new_paths
    
paths = search([])
while True:
    paths = search(paths)
    ends = set(x[-1] for x in paths)
    if len(ends) == 1 and 'end' in ends: break

#for p in paths:print(p)
    
output(len(paths)) #1

paths = search([])
while True:
    paths = search(paths, True)
    ends = set(x[-1] for x in paths)
    if len(ends) == 1 and 'end' in ends: break

#for p in paths:print(p)
    
output(len(paths)) #2

4775
152480


# Day 13

In [14]:
input = get_input(13)

dots = []
folds = []
for x in input:
    if not x:
        continue
    elif 'fold' in x:
        folds.append(x.split()[2].split('='))
    else:
        dots.append([int(c) for c in x.split(',')])
        
w = max([x[0] for x in dots]) + 1
h = max([x[1] for x in dots]) + 1
if w % 2 == 0: w += 1
if h % 2 == 0: h += 1

arr = np.zeros((h,w), np.int32)
for x,y in dots:
    arr[y][x] = 1

dots = arr

#print(dots)
for d, loc in folds:
    loc = int(loc)
    #print(d,loc,dots.shape,loc*2+1)
    if d == 'y':
        for y in range(loc):
            for x in range(w):
                dots[y][x] = dots[y][x] or dots[h-y-1][x]
        h = loc
        dots = dots[:h,:]
    else:
        for y in range(h):
            for x in range(loc):
                dots[y][x] = dots[y][x] or dots[y][w-x-1]
        w = loc
        dots = dots[:,:w]

    #print(dots)
    #break

output(np.sum(dots))
for row in dots:
    line = [str(x) for x in row]
    print(''.join(line).replace('1','#').replace('0',' '))

103
###  #  #  ##  #    ###   ##  ###   ##  
#  # #  # #  # #    #  # #  # #  # #  # 
#  # #### #  # #    #  # #    #  # #  # 
###  #  # #### #    ###  #    ###  #### 
# #  #  # #  # #    # #  #  # # #  #  # 
#  # #  # #  # #### #  #  ##  #  # #  # 


# Day 14